# Aux

In [6]:
import torch
import torch.optim as optim
#%pip install torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, TensorDataset
import numpy as np

#
import torch
import torch.nn as nn
import torch.optim as optim

#%pip install pennylane --user
import pennylane as qml
from pennylane import numpy as np

import sys
import os

src_path = os.path.abspath(os.path.join('..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

from circuits.convolution.default import default_circuit, full_entanglement_circuit
from circuits.convolution.NQ_circuit import NQ_circuit
from circuits.convolution.no_entanglement_circuit import no_entanglement_random_circuit
from circuits.embedding.default import default_embedding, custom_embedding
from circuits.embedding.IQP_embedding import custom_iqp_embedding
from circuits.embedding.NQE_embedding import NQE_embedding
from circuits.embedding.qml_embeddings import angle_embedding, amplitude_embedding, QAOA_embedding, displacement_embedding, squeezing_embedding
from circuits.measurement.default import default_measurement, custom_measurement
from layers.quanvolution import QuanvLayer
from models.quanvolutional_net import QuanvolutionalNet
from utils.data_loader import load_mnist_data

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Train

In [17]:
#%pip install qcml
#%pip install scikit-learn
#%pip install dask
#%pip install pynvml
#%pip install pandas
#%pip install matplotlib --user
#%pip install seaborn
#%pip install jax
from qcml.bench.grid_search import GridSearch
from qcml.utils.log import log_setup
import os

log_settings = {
    "output": "both",
    "terminal_level": "INFO",
    "file_level": "DEBUG",
    "hide_logs": ["jax", "pennylane", "bokeh", "distributed"],
    "slack_notify": False
}

log_setup(**log_settings)

X_train, y_train, X_val, y_val = load_mnist_data(batch_size=4, output='np', limit=500)

os.environ["JAX_PLATFORM_NAME"] = "cpu"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
torch.backends.cudnn.benchmark = True

2024-10-04 09:41:07,126 - root - INFO - Logging is set up. (log.py:391)
2024-10-04 09:41:07,128 - root - INFO - jax logs are set to ERROR level. (log.py:421)
2024-10-04 09:41:07,129 - root - INFO - pennylane logs are set to ERROR level. (log.py:421)
2024-10-04 09:41:07,131 - root - INFO - bokeh logs are set to ERROR level. (log.py:421)
2024-10-04 09:41:07,134 - root - INFO - distributed logs are set to ERROR level. (log.py:421)


Using device: cpu


In [18]:
import itertools

embedding_type = "angle"

qkernel_shape = [2, 3]
classical_kernel_shape = [2, 3, 5]

if embedding_type == "angle":

    # Angle embedding

    embedding = angle_embedding
    circuit = no_entanglement_random_circuit
    params = {}

    lrs = torch.linspace(0.009, 0.08, 3, requires_grad=False)
    learning_rates = [lr.item() for lr in lrs] 
    

elif embedding_type == "amplitude":

    # Amplitude embedding

    embedding = amplitude_embedding
    circuit = no_entanglement_random_circuit
    params = {}

    lrs = torch.linspace(0.008, 0.03, 3, requires_grad=False)
    learning_rates = [lr.item() for lr in lrs]
    

elif embedding_type == "IQP":

    # IQP embedding

    embedding = custom_iqp_embedding
    circuit = NQ_circuit
    params = {"embedding": {"n_repeats": 2}}

    lrs = torch.linspace(0.008, 0.05, 3, requires_grad=False)
    learning_rates = [lr.item() for lr in lrs]

elif embedding_type == "NQE":

    # NQE embedding

    embedding = NQE_embedding
    circuit = no_entanglement_random_circuit
    params = {"embedding": {"n_repeats": 2}}

    lrs = torch.linspace(0.008, 0.05, 3, requires_grad=False)
    learning_rates = [lr.item() for lr in lrs]

elif embedding_type == "displacement":

    # Displacement embedding

    embedding = angle_embedding
    circuit = no_entanglement_random_circuit
    params = {}

    lrs = torch.linspace(0.009, 0.08, 3, requires_grad=False)
    learning_rates = [lr.item() for lr in lrs]

elif embedding_type == "squeezing":

    # Squeezing embedding

    embedding = angle_embedding
    circuit = no_entanglement_random_circuit
    params = {}

    lrs = torch.linspace(0.009, 0.08, 3, requires_grad=False)
    learning_rates = [lr.item() for lr in lrs]

elif embedding_type == "QAOA":

    # Squeezing embedding

    embedding = QAOA_embedding
    circuit = NQ_circuit
    params = {"embedding": {"qkernel_shape": 3}}

    lrs = torch.linspace(0.009, 0.03, 3, requires_grad=False)
    learning_rates = [lr.item() for lr in lrs]

model_grid = {
        "qkernel_shape": qkernel_shape,
        "classical_kernel_shape": classical_kernel_shape,
        "embedding":[embedding],
        "circuit":[circuit],
        "measurement":[custom_measurement],
        "params":[params],
        "n_classes":[10],
        "learning_rate": learning_rates
    }

experiment_name = "quantum_mnist_" + embedding_type + ","

gs = GridSearch(
    classifiers=[QuanvolutionalNet],
    param_grid=model_grid,
    #combinations=combinations,
    batch_size=50,
    experiment_name=experiment_name
).run(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val)

2024-10-04 09:41:08,043 - qcml.bench.grid_search - INFO - Starting grid search with QuanvolutionalNet, 18 combinations, 12 parallel jobs (grid_search.py:165)


Epoch 1/10, Loss: 2.3016
Epoch 2/10, Loss: 2.2843
Epoch 3/10, Loss: 2.2478
Epoch 4/10, Loss: 2.1576
Epoch 5/10, Loss: 1.8805
Epoch 6/10, Loss: 1.6833
Epoch 7/10, Loss: 1.3598
Epoch 8/10, Loss: 1.3279
Epoch 9/10, Loss: 1.0481
Epoch 10/10, Loss: 1.1444


2024-10-04 09:41:12,462 - qcml.bench.grid_search - INFO - Evaluated params: {'qkernel_shape': 2, 'classical_kernel_shape': 2, 'embedding': <function angle_embedding at 0x0000028693484A60>, 'circuit': <function no_entanglement_random_circuit at 0x0000028693484310>, 'measurement': <function custom_measurement at 0x0000028693484550>, 'params': {}, 'n_classes': 10, 'learning_rate': 0.008999999612569809}, accuracy: 0.5200 (grid_search.py:241)


Epoch 1/10, Loss: 2.3196
Epoch 2/10, Loss: 2.2975
Epoch 3/10, Loss: 2.2947
Epoch 4/10, Loss: 2.2796
Epoch 5/10, Loss: 2.0352
Epoch 6/10, Loss: 1.7967
Epoch 7/10, Loss: 1.3107
Epoch 8/10, Loss: 1.2054
Epoch 9/10, Loss: 1.2073
Epoch 10/10, Loss: 1.1681


2024-10-04 09:41:16,597 - qcml.bench.grid_search - INFO - Evaluated params: {'qkernel_shape': 2, 'classical_kernel_shape': 2, 'embedding': <function angle_embedding at 0x0000028693484A60>, 'circuit': <function no_entanglement_random_circuit at 0x0000028693484310>, 'measurement': <function custom_measurement at 0x0000028693484550>, 'params': {}, 'n_classes': 10, 'learning_rate': 0.04450000077486038}, accuracy: 0.4900 (grid_search.py:241)


...

2024-10-04 09:44:15,239 - qcml.bench.grid_search - INFO - Evaluated params: {'qkernel_shape': 3, 'classical_kernel_shape': 5, 'embedding': <function angle_embedding at 0x0000028693484A60>, 'circuit': <function no_entanglement_random_circuit at 0x0000028693484310>, 'measurement': <function custom_measurement at 0x0000028693484550>, 'params': {}, 'n_classes': 10, 'learning_rate': 0.008999999612569809}, accuracy: 0.6660 (grid_search.py:241)


Epoch 1/10, Loss: 2.2795
Epoch 2/10, Loss: 2.0713
Epoch 3/10, Loss: 1.8989
Epoch 4/10, Loss: 1.6495
Epoch 5/10, Loss: 1.5256
Epoch 6/10, Loss: 1.4637
Epoch 7/10, Loss: 1.3663
Epoch 8/10, Loss: 1.2743
Epoch 9/10, Loss: 1.4231
Epoch 10/10, Loss: 1.1248


2024-10-04 09:44:38,932 - qcml.bench.grid_search - INFO - Evaluated params: {'qkernel_shape': 3, 'classical_kernel_shape': 5, 'embedding': <function angle_embedding at 0x0000028693484A60>, 'circuit': <function no_entanglement_random_circuit at 0x0000028693484310>, 'measurement': <function custom_measurement at 0x0000028693484550>, 'params': {}, 'n_classes': 10, 'learning_rate': 0.04450000077486038}, accuracy: 0.4380 (grid_search.py:241)


Epoch 1/10, Loss: 2.2871
Epoch 2/10, Loss: 2.0535
Epoch 3/10, Loss: 2.1262
Epoch 4/10, Loss: 2.1314
Epoch 5/10, Loss: 1.8967
Epoch 6/10, Loss: 1.9245
Epoch 7/10, Loss: 2.4521
Epoch 8/10, Loss: 2.0096
Epoch 9/10, Loss: 1.6896
Epoch 10/10, Loss: 1.5207


2024-10-04 09:45:00,571 - qcml.bench.grid_search - INFO - Evaluated params: {'qkernel_shape': 3, 'classical_kernel_shape': 5, 'embedding': <function angle_embedding at 0x0000028693484A60>, 'circuit': <function no_entanglement_random_circuit at 0x0000028693484310>, 'measurement': <function custom_measurement at 0x0000028693484550>, 'params': {}, 'n_classes': 10, 'learning_rate': 0.07999999821186066}, accuracy: 0.3660 (grid_search.py:241)
2024-10-04 09:45:00,576 - qcml.utils.storage - INFO - Results saved to results/grid_search/QuanvolutionalNet-quantum_mnist_angle,_best-hypa.csv, file size: 7907 bytes (storage.py:48)


In [7]:
model_grid_classical = {
    "classical_kernel_shape": [2, 3, 5],
    "n_classes": [10],
    "use_quantum": [False],
}

gs = GridSearch(
    classifiers=[QuanvolutionalNet],
    param_grid=model_grid_classical,
    #checkpoint_interval=1,
    batch_size=50,
    experiment_name="classical_mnist,"
).run(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val)


2024-09-30 09:15:21,877 - qcml.bench.grid_search - INFO - Starting grid search with QuanvolutionalNet, 3 combinations, 50 parallel jobs (grid_search.py:165)
2024-09-30 09:15:21,880 - qcml.bench.grid_search - ERROR - Error with parameters {'classical_kernel_shape': 2, 'n_classes': 10, 'use_quantum': False}: '<=' not supported between instances of 'float' and 'NoneType' (grid_search.py:246)


TypeError: '<=' not supported between instances of 'float' and 'NoneType'